# Contact Detection- user script

## Library module import

In [1]:
import sys

import numpy as np

In [2]:
np.seterr(all="ignore")

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [3]:
PYTHONTOOLSDIR = r"C:\Users\pramod.kumar\OneDrive - SIEMENSGAMESA\Work\ProjectPyAnsys"
sys.path.append(PYTHONTOOLSDIR)
import py_apdl_pre as pre

PyMAPDL is taking longer than expected to connect to an MAPDL session.
Checking if there are any available licenses...
Product:             Ansys Mechanical Enterprise
MAPDL Version:       22.2
ansys.mapdl Version: 0.65.2



In [4]:
PYTHONTOOLSDIR = r"C:\Users\pramod.kumar\OneDrive - SIEMENSGAMESA\Work\ContactDetect"
sys.path.append(PYTHONTOOLSDIR)
import ContactDetection as cd

In [5]:
mapdl = pre.mapdl

## Working directory setup

In [6]:
mapdl.db.clear()
dirpath = r"C:\Users\pramod.kumar\Music\contct_detect"
jobname = "MDT"
pre.mapdl.cwd(dirpath=dirpath)
pre.mapdl.filname(fname=jobname, key=1)
mapdl.title(title="Contact Detect Trial")
mapdl.prep7()

*** MAPDL - ENGINEERING ANALYSIS SYSTEM  RELEASE 2022 R2          22.2     ***
 Ansys Mechanical Enterprise                       
 00000000  VERSION=WINDOWS x64   16:07:57  SEP 05, 2023 CP=      0.656

 Contact Detect Trial                                                          



          ***** MAPDL ANALYSIS DEFINITION (PREP7) *****

In [7]:
print(mapdl.version)
print(mapdl.directory)
print(mapdl.jobname)

22.2
C:/Users/pramod.kumar/Music/contct_detect
MDT


## Parasolid import and component creation

In [8]:
parapath = (
    r"C:\Users\pramod.kumar\OneDrive - SIEMENSGAMESA\Work\ContactDetect\pyAnsysMAPDL"
)

In [9]:
cmname = "part1"
paraname = "Body1"
pre.paraimport(paraname, parapath, cmname)

Body1 imported as component part1 with vlum num [1]


In [10]:
cmname = "part2"
paraname = "Body2"
pre.paraimport(paraname, parapath, cmname)

Body2 imported as component part2 with vlum num [2]


In [11]:
cmname = "part3"
paraname = "Body3"
pre.paraimport(paraname, parapath, cmname)

Body3 imported as component part3 with vlum num [3]


In [12]:
cmname = "part4"
paraname = "Body4"
pre.paraimport(paraname, parapath, cmname)

Body4 imported as component part4 with vlum num [4]


In [13]:
cmname = "part5"
paraname = "Body5"
pre.paraimport(paraname, parapath, cmname)

Body5 imported as component part5 with vlum num [5]


In [14]:
cmname = "part6"
paraname = "Body6"
pre.paraimport(paraname, parapath, cmname)

Body6 imported as component part6 with vlum num [6]


In [15]:
mapdl.reset()
mapdl.replot()
mapdl.prep7()

You have already entered the general preprocessor (PREP7).

In [16]:
mapdl.cmlist(name="ALL", key=1, entity="VOLU")

LIST SELECTED COMPONENTS
 ENTITY TYPE = VOLU

 *** MAPDL - ENGINEERING ANALYSIS SYSTEM  RELEASE 2022 R2          22.2     ***
 Ansys Mechanical Enterprise                       
 00000000  VERSION=WINDOWS x64   16:08:18  SEP 05, 2023 CP=      2.906

  File: C:\Users\pramod.kumar\OneDrive - SIEMENSGAMESA\Work\ContactDetect\pyAns



    NAME                            SELE TYPE      SUBCOMPONENTS

  PART1                             NO   VOLU
                                               1
  PART2                             NO   VOLU
                                               2
  PART3                             NO   VOLU
                                               3
  PART4                             NO   VOLU
                                               4
  PART5                             NO   VOLU
                                               5
  PART6                             YES  VOLU
                                               6

In [17]:
mapdl.cmsel("ALL")

ALSO SELECT ALL COMPONENTS

In [18]:
mapdl.reset()
mapdl.replot()

In [19]:
# mapdl.view(wn="", xv="1", yv="1", zv="0")

In [20]:
mapdl.cmsel("ALL")
mapdl.vplot(color_areas=False, vtk=True, show_line_numbering=False, quality=2)


--------------------------------------------------------------------------------
   !!! You are currently using trame@3 which may break your application !!!
--------------------------------------------------------------------------------

 1. trame@3 only provides by default trame.widgets.[html,client] and remove
    everything else as implicit dependency. Those other widgets will still
    exist and will be supported, but they will need to be defined as a
    dependency of your application.

       $ pip install trame-vtk trame-vuetify trame-plotly

    Import paths are remaining the same.

    For libraries like vuetify since they offer different API between
    their vue2 and vue3 implementation, the widget name will reflect
    which vue version they are referencing. But original naming will remain.

       from trame.widgets import vuetify2, vuetify3


 2. trame@3 aims to use vue3 as a new default. But to smooth the transition
    we will maintain the server.client_type = 'vue2' 

Widget(value="<iframe src='http://localhost:55961/index.html?ui=P_0x1eea8aa34c0_0&reconnect=auto' style='width…

## Contact area detect

In [21]:
def find_contact_area(cm1, cm2):
    conta_targ = pre.contact_between_cm(cm1, cm2)
    print(conta_targ)
    area1 = list(conta_targ.keys())
    # print(area1)
    for area_1 in area1:
        for area_2 in conta_targ[area_1]:
            print(area_1, area_2)
            overlap, eq_area = pre.check_overlap(area_1, area_2)

            if overlap == 1:
                if eq_area == 1:
                    print(f"Complete overlap {area_1}, {area_2}")
                    break
                    # pass
                else:
                    print(f"centroid overlap ,(not area value) {area_1}, {area_2}")

            else:
                vect21, max_ind = pre.area21_kp_vector(area_1, area_2)
                l1, l2, l3, l4 = cd.vector_spred(vect21, max_ind)
                case1 = l1 * l2 * l3 * l4
                print(l1, l2, l3, l4 )

                vect21, max_ind = pre.area21_kp_vector(area_2, area_1)
                l1, l2, l3, l4 = cd.vector_spred(vect21, max_ind)
                case2 = l1 * l2 * l3 * l4
                print(l1, l2, l3, l4 )
                
                if case1 == 0 and case2 == 0:
                    print(f"no overlap {area_1}, {area_2}")

                if case1 != 0 and case2 == 0:
                    print(f"enclosed overlap {area_1}, {area_2}")

                if case1 == 0 and case2 != 0:
                    print(f"enclosed overlap {area_1}, {area_2}")

                if case1 != 0 and case2 != 0:
                    print(f"partial overlap {area_1}, {area_2}")

---

complete overlap

In [28]:
mapdl.cmsel(type_="S", name="PART3", entity="VOLU")
mapdl.cmsel(type_="A", name="PART4", entity="VOLU")
mapdl.allsel(labt="BELOW ", entity="VOLU")
mapdl.vplot(show_area_numbering=True,)

Widget(value="<iframe src='http://localhost:55961/index.html?ui=P_0x1eed18061c0_4&reconnect=auto' style='width…

In [29]:
cm1 = "PART3"
cm2 = "PART4"
find_contact_area(cm1, cm2)

{12: array([18]), 13: array([21]), 14: array([20]), 15: array([19]), 16: array([], dtype=int32), 17: array([22])}
12 18
4 33 0 4
3 0 65 15
no overlap 12, 18
13 21
56 0 0 93
15 9 0 16
no overlap 13, 21
14 20
164 139 0 130
69 263 0 8
no overlap 14, 20
15 19
50 31 8 202
5 72 0 246
enclosed overlap 15, 19
17 22
Complete overlap 17, 22


In [27]:
mapdl.cmsel(type_="S", name="PART2", entity="VOLU")
mapdl.cmsel(type_="A", name="PART1", entity="VOLU")
mapdl.allsel(labt="BELOW ", entity="VOLU")
mapdl.vplot()

Widget(value="<iframe src='http://localhost:55961/index.html?ui=P_0x1eec592b7c0_3&reconnect=auto' style='width…

In [26]:
cm1 = "PART1"
cm2 = "PART2"
find_contact_area(cm1, cm2)

{1: array([], dtype=int32), 2: array([], dtype=int32), 3: array([], dtype=int32), 4: array([ 5, 10])}
4 5
Complete overlap 4, 5


---

no overlap

In [26]:
mapdl.cmsel(type_="S", name="PART5", entity="VOLU")
mapdl.cmsel(type_="A", name="PART6", entity="VOLU")
mapdl.allsel(labt="BELOW ", entity="VOLU")
mapdl.vplot(color_areas=True, vtk=True)

Widget(value="<iframe src='http://localhost:62507/index.html?ui=P_0x20dd796e5e0_3&reconnect=auto' style='width…

In [27]:
cm1 = "PART6"
cm2 = "PART5"
find_contact_area(cm1, cm2)

{31: array([], dtype=int32), 32: array([], dtype=int32), 33: array([], dtype=int32), 34: array([], dtype=int32), 35: array([], dtype=int32), 36: array([], dtype=int32), 37: array([], dtype=int32)}


---

Partial overlap

In [19]:
mapdl.cmsel(type_="S", name="PART3", entity="VOLU")
mapdl.cmsel(type_="A", name="PART5", entity="VOLU")
mapdl.allsel(labt="BELOW ", entity="VOLU")
# mapdl.vplot(color_areas=True, vtk=True)
mapdl.vplot(show_area_numbering=True,
            show_line_numbering=True
           )

Widget(value="<iframe src='http://localhost:53572/index.html?ui=P_0x2ebef83b670_1&reconnect=auto' style='width…

In [25]:
cm1 = "PART5"
cm2 = "PART3"
find_contact_area(cm1, cm2)

{24: array([], dtype=int32), 25: array([], dtype=int32), 26: array([], dtype=int32), 27: array([14]), 28: array([], dtype=int32), 29: array([], dtype=int32), 30: array([15])}
27 14
51 0 64 24
30 0 9 86
no overlap 27, 14
30 15
73 2 20 8
10 261 1 7
partial overlap 30, 15


In [22]:
mapdl.cmsel(type_="S", name="PART5", entity="VOLU")
mapdl.cmsel(type_="A", name="PART4", entity="VOLU")
# mapdl.cmsel(type_="A", name="PART4", entity="VOLU")
mapdl.allsel(labt="BELOW ", entity="VOLU")
# mapdl.vplot(color_areas=True, vtk=True)
mapdl.vplot(
    # show_area_numbering=True,
            show_line_numbering=True
           )

Widget(value="<iframe src='http://localhost:55961/index.html?ui=P_0x1eeb40ec7f0_1&reconnect=auto' style='width…

In [24]:
mapdl.cmsel(type_="S", name="PART5", entity="VOLU")
mapdl.cmsel(type_="A", name="PART4", entity="VOLU")
# mapdl.cmsel(type_="A", name="PART4", entity="VOLU")
mapdl.allsel(labt="BELOW ", entity="VOLU")
# mapdl.vplot(color_areas=True, vtk=True)
mapdl.vplot(
    # show_area_numbering=True,
            show_line_numbering=True
           )

Widget(value="<iframe src='http://localhost:55961/index.html?ui=P_0x1eec3fdf250_2&reconnect=auto' style='width…

In [35]:
# mapdl.ldiv(732, ndiv=10)

DIVIDE LINE    732 INTO      10 LINES

 NUMBER OF LINES DIVIDED =      1

In [33]:
# mapdl.lesize(nl1=732, size=0.1250E-03)

SET DIVISIONS ON LINE   732 (IF SELECTED)
      FOR ELEMENT SIZE =  0.12500E-03    SPACING RATIO =     1.0000

In [23]:
cm1 = "PART5"
cm2 = "PART4"
find_contact_area(cm1, cm2)

{24: array([], dtype=int32), 25: array([], dtype=int32), 26: array([], dtype=int32), 27: array([20]), 28: array([], dtype=int32), 29: array([], dtype=int32), 30: array([19])}
27 20
27 0 34 18
88 10 0 27
no overlap 27, 20
30 19
73 2 20 8
15 205 57 2
partial overlap 30, 19


---

In [25]:
mapdl.exit()